In [ ]:
import nibabel as nib
import numpy as np
import cc3d

input_nifti_path = r"[path]"
output_nifti_path =  r"[path]"

nifti_img = nib.load(input_nifti_path)
label_data = nifti_img.get_fdata().astype(np.uint8)

labeled_data = cc3d.connected_components(label_data, connectivity=26) 

new_nifti = nib.Nifti1Image(labeled_data.astype(np.int16), affine=nifti_img.affine)
nib.save(new_nifti, output_nifti_path)

print(f"Connected components applied and saved to {output_nifti_path}")

In [ ]:
nifti_img = nib.load(input_nifti_path)
label_data = nifti_img.get_fdata()

print("Unique Labels:", np.unique(label_data))
print("Data Type:", label_data.dtype)
print("Shape:", label_data.shape)


In [ ]:
label_data = label_data.astype(np.uint16)

In [ ]:
import cc3d

processed_labels = np.zeros_like(label_data, dtype=np.uint16)

for label in np.unique(label_data):
    if label == 0:
        continue
    mask = (label_data == label)
    cc_labels = cc3d.connected_components(mask.astype(np.uint8), connectivity=18) 
    processed_labels[mask] = cc_labels[mask] + (processed_labels.max() + 1)

output_nifti = nib.Nifti1Image(processed_labels, affine=nifti_img.affine)
nib.save(output_nifti, output_nifti_path)
print("Processed NIFTI saved!")

In [ ]:
stats = cc3d.statistics(processed_labels)

for i, size in enumerate(stats['voxel_counts']):
    if size < 100000000:
        processed_labels[processed_labels == i] = 0
        

output_nifti = nib.Nifti1Image(processed_labels, affine=nifti_img.affine)
nib.save(output_nifti, output_nifti_path)
print("Processed NIFTI saved!")

In [ ]:
import nibabel as nib
import numpy as np
import cc3d

input_nifti_path = r"[path]"
output_nifti_path =  r"[path]"


nifti_img = nib.load(input_nifti_path)
label_data = nifti_img.get_fdata()

print("Unique Labels:", np.unique(label_data))
print("Data Type:", label_data.dtype)
print("Shape:", label_data.shape)

label_data = label_data.astype(np.uint16)

processed_labels = np.zeros_like(label_data, dtype=np.uint16)

for label in np.unique(label_data):
    if label == 0:
        continue
    
    mask = (label_data == label) 
    cc_labels = cc3d.connected_components(mask.astype(np.uint8), connectivity=18)
    
    stats = cc3d.statistics(cc_labels)
    
    component_sizes = stats['voxel_counts'][1:] 
    if len(component_sizes) == 0:
        continue

    largest_component_index = np.argmax(component_sizes) + 1
    
    largest_component_mask = (cc_labels == largest_component_index)
    processed_labels[largest_component_mask] = label 

output_nifti = nib.Nifti1Image(processed_labels, affine=nifti_img.affine)
nib.save(output_nifti, output_nifti_path)
print(f"Largest connected components saved to {output_nifti_path}!")


# Connected Components Processing

In [ ]:
import os
import nibabel as nib
import numpy as np
import cc3d

def load_nifti(file_path):
    """Load a NIFTI file and return the image and data array."""
    nifti_img = nib.load(file_path)
    data = nifti_img.get_fdata().astype(np.uint16)
    return nifti_img, data

def save_nifti(data, reference_nifti, output_path):
    """Save a NIFTI file using a reference affine."""
    output_nifti = nib.Nifti1Image(data, affine=reference_nifti.affine)
    nib.save(output_nifti, output_path)

def process_largest_connected_component(label_data):
    """Keep only the largest connected component for each unique label."""
    processed_labels = np.zeros_like(label_data, dtype=np.uint16)

    for label in np.unique(label_data):
        if label == 0:
            continue

        mask = (label_data == label)
        cc_labels = cc3d.connected_components(mask.astype(np.uint8), connectivity=18)
        stats = cc3d.statistics(cc_labels)

        component_sizes = stats['voxel_counts'][1:] 
        if len(component_sizes) == 0:
            continue

        largest_component_index = np.argmax(component_sizes) + 1
        largest_component_mask = (cc_labels == largest_component_index)

        processed_labels[largest_component_mask] = label  

    return processed_labels

def process_nifti_directory(input_dir, output_dir):
    """Process all NIFTI files in a directory and save the largest components."""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir) 

    nifti_files = [f for f in os.listdir(input_dir) if f.endswith(".nii") or f.endswith(".nii.gz")]

    for file_name in nifti_files:
        input_path = os.path.join(input_dir, file_name)
        output_path = os.path.join(output_dir, file_name)

        print(f"Processing: {file_name}")

        try:
            nifti_img, label_data = load_nifti(input_path)
            processed_data = process_largest_connected_component(label_data)
            save_nifti(processed_data, nifti_img, output_path)
            print(f"Saved: {output_path}")
        except Exception as e:
            print(f"Error processing {file_name}: {e}")

input_directory = r"[path]"
output_directory = r"[path]"

process_nifti_directory(input_directory, output_directory)

# Historgram Generation for Connected Components

In [ ]:
import os
import numpy as np
import nibabel as nib
import cc3d
import matplotlib.pyplot as plt
import seaborn as sns

def process_largest_connected_component(label_data):
    """Extract the largest connected component from a NIFTI label."""
    cc_labels = cc3d.connected_components(label_data, connectivity=18)
    stats = cc3d.statistics(cc_labels)
    sizes = stats['voxel_counts'][1:]
    
    if len(sizes) == 0:
        return np.zeros_like(label_data) 
    
    largest_component = np.argmax(sizes) + 1 
    return (cc_labels == largest_component).astype(np.uint16)

def create_summary_visualization(input_dir, output_dir, viz_output_path):
    """Generate a single visualization summarizing the entire dataset."""
    nifti_files = [f for f in os.listdir(input_dir) if f.endswith(".nii") or f.endswith(".nii.gz")]

    original_sizes = []
    processed_sizes = []
    example_original = None
    example_processed = None

    for file_name in nifti_files:
        input_path = os.path.join(input_dir, file_name)
        output_path = os.path.join(output_dir, file_name)

        nifti_img = nib.load(input_path)
        label_data = nifti_img.get_fdata().astype(np.uint16)

        processed_data = process_largest_connected_component(label_data)

        cc_labels = cc3d.connected_components(label_data, connectivity=18)
        stats = cc3d.statistics(cc_labels)
        original_sizes.extend(stats['voxel_counts'][1:])
        
        largest_size = np.sum(processed_data > 0)
        if largest_size > 0:
            processed_sizes.append(largest_size)

        if example_original is None:
            mid_slice = label_data.shape[2] // 2
            example_original = label_data[:, :, mid_slice]
            example_processed = processed_data[:, :, mid_slice]

    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    axes[0].imshow(example_original, cmap="gray")
    axes[0].set_title("Original Slice")
    axes[0].axis("off")

    axes[1].imshow(example_processed, cmap="gray")
    axes[1].set_title("Processed Slice")
    axes[1].axis("off")

    sns.histplot(original_sizes, bins=30, color="red", label="Before Processing", ax=axes[2], kde=True)
    sns.histplot(processed_sizes, bins=30, color="blue", label="After Processing", ax=axes[2], kde=True)
    axes[2].set_xlabel("Component Size (voxels)")
    axes[2].set_ylabel("Frequency")
    axes[2].set_title("Connected Component Size Distribution")
    axes[2].legend()

    plt.tight_layout()
    plt.savefig(viz_output_path)
    plt.close()
    print(f"Saved summary visualization: {viz_output_path}")

input_directory = r"[path]"
output_directory = r"[path]"
visualization_directory = r"[path]/graph.png"

create_summary_visualization(input_directory, output_directory, visualization_directory)

In [ ]:
import os
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

def calculate_voxel_count(nifti_file):
    """Calculate total voxels for a given NIFTI file."""
    nifti_img = nib.load(nifti_file)
    dims = nifti_img.shape
    total_voxels = dims[0] * dims[1] * dims[2]
    
    if len(dims) == 4:
        total_voxels *= dims[3]
    
    return total_voxels

def process_nifti_folder(folder_path):
    """Process all NIFTI files in the folder and calculate total voxels."""
    nifti_files = [f for f in os.listdir(folder_path) if f.endswith('.nii') or f.endswith('.nii.gz')]
    nifti_files = nifti_files[:50]
    results = []
    
    for nifti_file in nifti_files:
        nifti_path = os.path.join(folder_path, nifti_file)
        total_voxels = calculate_voxel_count(nifti_path)
        results.append({'File Name': nifti_file, 'Total Voxels': total_voxels})
    
    return results

def plot_voxel_comparison(before_folder_path, after_folder_path):
    """Generate a bar plot comparing voxel counts before and after processing."""

    before_results = process_nifti_folder(before_folder_path)
    
    after_results = process_nifti_folder(after_folder_path)
    
    df_before = pd.DataFrame(before_results)
    df_after = pd.DataFrame(after_results)
    
    comparison_df = pd.merge(df_before, df_after, on='File Name', suffixes=('_Before_Processing', '_After_Processing'))
    
    fig, ax = plt.subplots(figsize=(12, 8))
    comparison_df.plot.bar(x='File Name', y=['Total Voxels_Before_Processing', 'Total Voxels_After_Processing'],
                           ax=ax, width=0.8, color=['blue', 'green'])
    
    ax.set_xlabel('NIFTI Image')
    ax.set_ylabel('Voxel Count')
    ax.set_title('Voxel Count Comparison Before and After Processing')
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.show()

def plot_voxel_scatter(before_folder_path, after_folder_path):
    """Generate a scatter plot comparing voxel counts before and after processing."""

    before_results = process_nifti_folder(before_folder_path)
    
    after_results = process_nifti_folder(after_folder_path)
    
    df_before = pd.DataFrame(before_results)
    df_after = pd.DataFrame(after_results)
    
    comparison_df = pd.merge(df_before, df_after, on='File Name', suffixes=('_Before_Processing', '_After_Processing'))
    
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.scatter(comparison_df['Total Voxels_Before_Processing'], comparison_df['Total Voxels_After_Processing'], color='blue')
    
    ax.set_xlabel('Total Voxels Before Processing')
    ax.set_ylabel('Total Voxels After Processing')
    ax.set_title('Voxel Count Scatter Plot Before vs After Processing')
    
    ax.plot([comparison_df['Total Voxels_Before_Processing'].min(), comparison_df['Total Voxels_Before_Processing'].max()],
            [comparison_df['Total Voxels_After_Processing'].min(), comparison_df['Total Voxels_After_Processing'].max()],
            color='red', linestyle='--')
    
    plt.tight_layout()
    plt.show()


before_folder_path = r"[path]"
after_folder_path = r"[path]"


plot_voxel_comparison(before_folder_path, after_folder_path)

plot_voxel_scatter(before_folder_path, after_folder_path)


In [ ]:
import os
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

def calculate_mean_intensity(nifti_file):
    """Calculate mean intensity for a given NIFTI file."""
    nifti_img = nib.load(nifti_file)
    data = nifti_img.get_fdata()
    
    mean_intensity = np.mean(data)
    return mean_intensity

def process_nifti_folder_for_intensity(folder_path):
    """Process all NIFTI files in the folder and calculate mean intensities."""
    nifti_files = [f for f in os.listdir(folder_path) if f.endswith('.nii') or f.endswith('.nii.gz')]
    
    nifti_files = nifti_files[:50]
    
    results = []
    
    for nifti_file in nifti_files:
        nifti_path = os.path.join(folder_path, nifti_file)
        mean_intensity = calculate_mean_intensity(nifti_path)
        results.append({'File Name': nifti_file, 'Mean Intensity': mean_intensity})
    
    return results

def plot_intensity_comparison(before_folder_path, after_folder_path):
    """Generate a bar plot comparing intensity distributions before and after processing."""

    before_results = process_nifti_folder_for_intensity(before_folder_path)
    
    after_results = process_nifti_folder_for_intensity(after_folder_path)
    
    df_before = pd.DataFrame(before_results)
    df_after = pd.DataFrame(after_results)
    
    comparison_df = pd.merge(df_before, df_after, on='File Name', suffixes=('_Before_Processing', '_After_Processing'))
    
    fig, ax = plt.subplots(figsize=(12, 8))
    comparison_df.plot.bar(x='File Name', y=['Mean Intensity_Before_Processing', 'Mean Intensity_After_Processing'],
                           ax=ax, width=0.8, color=['blue', 'green'])
    
    ax.set_xlabel('NIFTI Image')
    ax.set_ylabel('Mean Intensity')
    ax.set_title('Mean Intensity Comparison Before and After Processing')
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.show()

def plot_intensity_scatter(before_folder_path, after_folder_path):
    """Generate a scatter plot comparing intensity values before and after processing."""

    before_results = process_nifti_folder_for_intensity(before_folder_path)
    
    after_results = process_nifti_folder_for_intensity(after_folder_path)
    
    df_before = pd.DataFrame(before_results)
    df_after = pd.DataFrame(after_results)
    
    comparison_df = pd.merge(df_before, df_after, on='File Name', suffixes=('_Before_Processing', '_After_Processing'))
    
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.scatter(comparison_df['Mean Intensity_Before_Processing'], comparison_df['Mean Intensity_After_Processing'], color='blue')
    
    ax.set_xlabel('Mean Intensity Before Processing')
    ax.set_ylabel('Mean Intensity After Processing')
    ax.set_title('Intensity Scatter Plot Before vs After Processing')
    
    ax.plot([comparison_df['Mean Intensity_Before_Processing'].min(), comparison_df['Mean Intensity_Before_Processing'].max()],
            [comparison_df['Mean Intensity_After_Processing'].min(), comparison_df['Mean Intensity_After_Processing'].max()],
            color='red', linestyle='--')
    
    plt.tight_layout()
    plt.show()

before_folder_path = r"[path]"
after_folder_path = r"[path]"


plot_intensity_comparison(before_folder_path, after_folder_path)

plot_intensity_scatter(before_folder_path, after_folder_path)
